In [19]:
import os
import subprocess
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib.colors import LogNorm
#import flopy

import warnings



#sns.set_theme(style="darkgrid")
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams.update({'font.size': 11})

In [2]:
org_model_ws = os.path.join('..', '2022_03_16_calib') # WS of calibrated model
ml_ws = os.path.join('..', 'temp_ml_ies') # New WS

In [3]:
if os.path.exists(ml_ws):
    shutil.rmtree(ml_ws)

os.mkdir(ml_ws)

In [4]:
shutil.copytree(org_model_ws, ml_ws, dirs_exist_ok=True)

'..\\temp_ml_ies'

In [5]:
#shutil.rmtree(os.path.join(ml_ws, 'agent'))

In [6]:
pest_ws = os.path.join(ml_ws, 'pest')

In [7]:
# Clean up directory
#for file in os.listdir(pest_ws):
#    if 'calib1-wt-cov' in file:
#        os.remove(os.path.join(pest_ws, file))

In [8]:
# objective function reduction
ofr = pd.read_csv(os.path.join(pest_ws, 'calib1-wt-cov-linux.ofr'), skiprows=3, delim_whitespace=True)

In [9]:
ofr

iteration  measurement  regularisation     heads   stage1     gage1  \
0          0      8999.96    3.169282e-11  999.9800  999.983  1000.000   
1          1      3153.42    7.366530e+03  304.3270  293.235   316.414   
2          2      2341.48    1.858790e+04  188.5080  328.941   245.321   
3          3      2090.77    4.176340e+04  169.1080  195.128   234.355   
4          4      2042.27    4.864360e+04  159.1450  192.721   241.557   
5          5      1864.93    7.318830e+04   71.0270  149.136   245.807   
6          6      1840.29    7.554060e+04   69.5048  147.694   244.731   
7          7      1835.14    7.832730e+04   68.2795  145.928   245.156   
8          8      1835.14    7.832730e+04   68.2795  145.928   245.156   

     gage2  dh_hedstg     >@dts  >@of   heads_d  stage1_d   gage1_d  gage2_d  \
0  999.998  1000.0000   0.00000   0.0  1000.000  1000.000  1000.000  999.999   
1  265.983   209.5990   0.00000   0.0   632.637   769.909   234.062  127.249   
2  175.858   114.6120  21.76280   0.0   511.841   463.390   186.365  104.879   
3  151.283   101.2610   3.42874   0.0   499.110   454.071   179.104  103.916   
4  145.805    90.2431   5.86858   0.0   494.076   426.091   183.868  102.891   
5  153.773    63.7210   8.97896   0.0   472.664   409.247   187.516  103.061   
6  145.746    62.1535   2.14632   0.0   467.007   410.619   187.642  103.050   
7  147.425    61.8901   1.09468   0.0   465.557   408.944   188.017  102.850   
8  147.425    61.8901   1.09468   0.0   465.557   408.944   188.017  102.850   

   pfluxb  pofb  pfluxr  pofr  
0     0.0   0.0     0.0   0.0  
1     0.0   0.0     0.0   0.0  
2     0.0   0.0     0.0   0.0  
3     0.0   0.0     0.0   0.0  
4     0.0   0.0     0.0   0.0  
5     0.0   0.0     0.0   0.0  
6     0.0   0.0     0.0   0.0  
7     0.0   0.0     0.0   0.0  
8     0.0   0.0     0.0   0.0

In [12]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8), dpi=90)

axs[0].plot(ofr['measurement'], label='Measurement objective function')
axs[0].set_ylim([0, 10000])
axs[0].legend()

for col in ofr.columns:
    if col in ['measurement', 'iteration', 'regularisation']:
        continue
    else:
        axs[1].plot(ofr[col], label=col)
        axs[1].legend(fontsize=8)
        
    
plt.xlabel('Iteration')
plt.ylabel('Objective function value')
plt.legend()
plt.show()

### Run parrep

In [13]:
p = subprocess.run([
    'parrep',
    'calib1-wt-cov-linux.par.5', # CHANGE THIS
    'calib1-wt-cov-linux.pst',
    'calib-parrep.pst',
    '0'
], cwd=pest_ws, stdout=subprocess.PIPE, text=True)

for row in p.stdout.split('\n'):
    print(row)


PARREP Version 17.3. Watermark Numerical Computing.

Reading parameter value file calib1-wt-cov-linux.par.5 ----->
Data for 1719 parameters read from file calib1-wt-cov-linux.par.5.

Reading file calib1-wt-cov-linux.pst and writing file calib-parrep.pst ----->
File calib-parrep.pst written ok.



### Generate observation realizations

In [14]:
p = subprocess.run(
    ['subreg1', 'calib-parrep.pst', 'ies-postcalib.pst'],
    cwd=pest_ws,
    stdout=subprocess.PIPE,
    text=True
)
for row in p.stdout.split('\n'):
    print(row)


 SUBREG1 version 17.3. Watermark Numerical Computing.

 - file calib-parrep.pst read ok.
 - file ies-postcalib.pst written ok.



In [15]:
# Clean up directory
for file in os.listdir(pest_ws):
    if 'calib-parrep' in file:
        os.remove(os.path.join(pest_ws, file))

In [16]:
# Clean up directory
for file in os.listdir(pest_ws):
    if 'calib1-wt-cov-linux' in file:
        os.remove(os.path.join(pest_ws, file))

### Create parameter uncertainty file

In [46]:
def write_script(filename, lines):

    with open(filename, 'a') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

In [70]:
# variance multiplier (given log transformed parameter - for use with covariance matrices)
# np.var(np.log10([values, of, parameters]))
np.var(np.log10([0.1, 1, 10]))

0.6666666666666666

In [68]:
# stdev multiplier (given log transformed parameter)
# np.std(np.log10([lbnd, ubnd]))
np.std(np.log10([0.1, 1])) # Lumprem offset (?)

0.5

In [47]:
write_script(os.path.join(pest_ws, 'param.unc'), [
f'''
### HYDRAULIC CONDUCTIVITY ###

# Kh # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_cond.mat"
first_parameter khpp001
last_parameter khpp680
variance_multiplier 0.25
END COVARIANCE_MATRIX


### STORAGE ###

# Sy # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sy_rchmul.mat"
first_parameter sypp001
last_parameter sypp221
variance_multiplier 0.25
END COVARIANCE_MATRIX

### RCH Multiplier ###

# Rchmul # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sy_rchmul.mat"
first_parameter rchmulpp001
last_parameter rchmulpp221
variance_multiplier 0.25
END COVARIANCE_MATRIX

### GENERAL HEAD BOUNDARIES ###

# GHB Red # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_ghb_red.mat"
first_parameter ghbcredpp01
last_parameter ghbcredpp13
variance_multiplier 0.25
END COVARIANCE_MATRIX

# GHB Yellow # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_ghb_yellow.mat"
first_parameter ghbcyelpp01
last_parameter ghbcyelpp18
variance_multiplier 0.25
END COVARIANCE_MATRIX

# GHB Green # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_ghb_green.mat"
first_parameter ghbcgrepp01
last_parameter ghbcgrepp15
variance_multiplier 0.25
END COVARIANCE_MATRIX

# GHB Blue # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_ghb_blue.mat"
first_parameter ghbcblupp01
last_parameter ghbcblupp19
variance_multiplier 0.25
END COVARIANCE_MATRIX

### STREAMFLOW ROUTING ###

# SFR man # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr.mat"
first_parameter sfrppman001
last_parameter sfrppman053
variance_multiplier 0.25
END COVARIANCE_MATRIX

# SFR rbth # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr.mat"
first_parameter sfrpprbth001
last_parameter sfrpprbth053
variance_multiplier 0.25
END COVARIANCE_MATRIX

# SFR rgrd # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr.mat"
first_parameter sfrpprgrd001
last_parameter sfrpprgrd053
variance_multiplier 0.25
END COVARIANCE_MATRIX

# SFR rwid # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr.mat"
first_parameter sfrpprwid001
last_parameter sfrpprwid053
variance_multiplier 0.25
END COVARIANCE_MATRIX

# SFR rhk # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr.mat"
first_parameter sfrpprhk001
last_parameter sfrpprhk053
variance_multiplier 0.25
END COVARIANCE_MATRIX

# SFR reach 1 inflow # OK
START COVARIANCE_MATRIX
FILE "..\\runmodel\\preproc\\cov_sfr_inf1.mat"
first_parameter sfrinflow000
last_parameter sfrinflow212
variance_multiplier 0.25
END COVARIANCE_MATRIX

### NON COVARIANCE MATRIX PARAMETERS ###

# SFR rtp
START STANDARD_DEVIATION
sfrrtpexcoef 1.0 # OK
sfrrtpoffset 1.15 # OK
sfrrtpsigma 1.25 # OK
END STANDARD_DEVIATION

# SFR reach evaporation and rainfall multipliers
START STANDARD_DEVIATION
sfr_ev_mul 1.0 # OK
sfr_rf_mul 1.0 # OK
END STANDARD_DEVIATION

# GHB yellow and blue multipliers
START STANDARD_DEVIATION
ghbyellomult 1.5 # OK
ghbbluemult 1.5 # OK
END STANDARD_DEVIATION

# Lumprem crop factor
START STANDARD_DEVIATION
crfac_lu1 1.5 # OK
crfac_red 1.5 # OK
crfac_yel 1.5 # OK
crfac_gre 1.5 # OK
crfac_blu 1.5 # OK
END STANDARD_DEVIATION

# Lumprem gamma
START STANDARD_DEVIATION
gamma_lu1 2.0 # OK
gamma_red 2.0 # OK
gamma_yel 2.0 # OK
gamma_gre 2.0 # OK
gamma_blu 2.0 # OK
END STANDARD_DEVIATION

# Lumprem ks
START STANDARD_DEVIATION
ks_lu1 1.5 # OK
ks_red 1.5 # OK
ks_yel 1.5 # OK
ks_gre 1.5 # OK
ks_blu 1.5 # OK
END STANDARD_DEVIATION

# Lumprem m
START STANDARD_DEVIATION
m_lu1 1.15 # OK
m_red 1.15 # OK
m_yel 1.15 # OK
m_gre 1.15 # OK
m_blu 1.15 # OK
END STANDARD_DEVIATION

# Lumprem maxvol
START STANDARD_DEVIATION
mvol_lu1 0.25 # OK
mvol_red 0.25 # OK
mvol_yel 0.25 # OK
mvol_gre 0.25 # OK
mvol_blu 0.25 # OK
END STANDARD_DEVIATION

# Lumprem initial volume
START STANDARD_DEVIATION
vol_lu1 1.0 # OK
vol_red 1.0 # OK
vol_yel 1.0 # OK
vol_gre 1.0 # OK
vol_blu 1.0 # OK
END STANDARD_DEVIATION

# Lumprem factor 1 (for GHB models)
START STANDARD_DEVIATION
f1_red 1.5 # OK
f1_yel 1.5 # OK
f1_gre 1.5 # OK
f1_blu 1.5 # OK
END STANDARD_DEVIATION

# Lumprem factor 2 (for GHB models)
START STANDARD_DEVIATION
f2_red 2.0 # OK
f2_yel 2.0 # OK
f2_gre 2.0 # OK
f2_blu 2.0 # OK
END STANDARD_DEVIATION

# Lumprem power (for GHB models)
START STANDARD_DEVIATION
p_red 2.0 # OK
p_yel 2.0 # OK
p_gre 2.0 # OK
p_blu 2.0 # OK
END STANDARD_DEVIATION

# Lumprem offset (for GHB models)
START STANDARD_DEVIATION
of_red 1.0 # OK
of_yel 1.0 # OK
of_gre 1.0 # OK
of_blu 1.0 # OK
END STANDARD_DEVIATION
'''
])

### Set weights according to standard deviation of measurement noise 

In [48]:
with open(os.path.join(pest_ws, 'ies-postcalib.pst'), 'r') as f:
    pstfile = f.readlines()

obs_strt = pstfile.index('* observation data\n') + 1
obs_stop = pstfile.index('* model command line\n')

In [49]:
df = pd.DataFrame(
    [i.split() for i in pstfile[obs_strt:obs_stop]],
    columns=['OBSNME', 'OBSVAL', 'WEIGHT', 'OBGNME']
)
df

OBSNME     OBSVAL    WEIGHT OBGNME
0          b10-1_1   133.6950  0.834116  heads
1          b10-1_2   133.6200  0.834116  heads
2          b10-1_3   133.4800  0.834116  heads
3          b10-1_4   133.4250  0.834116  heads
4          b14-1_1   140.2000  0.834116  heads
...            ...        ...       ...    ...
172894  of-262_209  -20587.87   0.00000   pofr
172895  of-262_210  -20814.32   0.00000   pofr
172896  of-262_211  -20974.86   0.00000   pofr
172897  of-262_212  -20827.10   0.00000   pofr
172898  of-262_213  -20648.60   0.00000   pofr

[172899 rows x 4 columns]

In [50]:
df.OBGNME.unique()

array(['heads', 'stage1', 'gage1', 'gage2', 'dh_hedstg', '>@dts', '>@of',
       'heads_d', 'stage1_d', 'gage1_d', 'gage2_d', 'pfluxb', 'pofb',
       'pfluxr', 'pofr'], dtype=object)

In [51]:
# Set gage1 measurement error at 25% of measured value (for all non-zero obs):
gage1_weights = [round(1/(float(i) * .25) ,6) for i in df.loc[(df['OBGNME'] == 'gage1') & (df['WEIGHT'] != '0.00000')].OBSVAL.values]
gage1_d_weights = [abs(round(1/(float(i) * .025) ,6)) for i in df.loc[(df['OBGNME'] == 'gage1_d') & (df['WEIGHT'] != '0.00000')].OBSVAL.values]

In [52]:
#Set gage2 measurement error at 10% of measured value (for all non-zero obs):
gage2_weights = [round(1/(float(i) * .1), 6) for i in df.loc[df['OBGNME'] == 'gage2'].OBSVAL.values]
gage2_d_weights = [abs(round(1/(float(i) * .01), 6)) for i in df.loc[df['OBGNME'] == 'gage2_d'].OBSVAL.values]

In [53]:
# Set weights to the inverse of the standard deviation of measurement noise
df.loc[df['OBGNME'] == 'heads', 'WEIGHT'] = 1 / 0.1 # stdev of head measurement noise (10 cm)
df.loc[df['OBGNME'] == 'heads_d', 'WEIGHT'] = 1 / 0.01 # stdev of head differences measurement noise (1cm)
df.loc[df['OBGNME'] == 'stage1', 'WEIGHT'] = 1 / 0.1 # stdev of stage measurement noise (10cm)
df.loc[df['OBGNME'] == 'stage1_d', 'WEIGHT'] = 1 / 0.015 # stdev of stage differences measurement noise (1cm)
df.loc[df['OBGNME'] == 'dh_hedstg', 'WEIGHT'] = 1 / 0.01 # stdev of head/stage differences measurement noise (1cm)
df.loc[(df['OBGNME'] == 'gage1') & (df['WEIGHT'] != '0.00000'), 'WEIGHT'] = gage1_weights
df.loc[(df['OBGNME'] == 'gage1_d') & (df['WEIGHT'] != '0.00000'), 'WEIGHT'] = gage1_d_weights
df.loc[df['OBGNME'] == 'gage2', 'WEIGHT'] = gage2_weights
df.loc[df['OBGNME'] == 'gage2_d', 'WEIGHT'] = gage2_d_weights
df.loc[df['OBGNME'] == '>@dts', 'WEIGHT'] = 0 # No measurement noise
df.loc[df['OBGNME'] == '>@of', 'WEIGHT'] = 0 # No measurement noise

In [54]:
df['OBGNME'] = df['OBGNME'] + '\n'

In [55]:
obsdata = ['* observation data\n']
for i in df.values.tolist():
    obsdata.append(' '.join(str(v) for v in i))

In [56]:
obsdata

['* observation data\n',
 'b10-1_1 133.6950 10.0 heads\n',
 'b10-1_2 133.6200 10.0 heads\n',
 'b10-1_3 133.4800 10.0 heads\n',
 'b10-1_4 133.4250 10.0 heads\n',
 'b14-1_1 140.2000 10.0 heads\n',
 'b14-1_2 139.9000 10.0 heads\n',
 'b14-1_3 139.8800 10.0 heads\n',
 'b15-1_1 141.2040 10.0 heads\n',
 'b15-1_2 140.9400 10.0 heads\n',
 'b15-1_3 140.9400 10.0 heads\n',
 'b16-1_1 138.0810 10.0 heads\n',
 'b16-1_2 137.8700 10.0 heads\n',
 'b16-1_3 137.8200 10.0 heads\n',
 'b2-1_1 137.0535 10.0 heads\n',
 'b2-1_2 137.0488 10.0 heads\n',
 'b2-1_3 137.0533 10.0 heads\n',
 'b2-1_4 137.0538 10.0 heads\n',
 'b2-1_5 137.0558 10.0 heads\n',
 'b2-1_6 137.0580 10.0 heads\n',
 'b2-1_7 137.0651 10.0 heads\n',
 'b2-1_8 137.0673 10.0 heads\n',
 'b2-1_9 137.0738 10.0 heads\n',
 'b2-1_10 137.0710 10.0 heads\n',
 'b2-1_11 137.0715 10.0 heads\n',
 'b2-1_12 137.0782 10.0 heads\n',
 'b2-1_13 137.0688 10.0 heads\n',
 'b2-1_14 137.0549 10.0 heads\n',
 'b2-1_15 137.0578 10.0 heads\n',
 'b2-1_16 137.0608 10.0 heads\n'

In [57]:
with open(os.path.join(pest_ws, 'obsweights.dat'), 'w') as f:
    for line in obsdata:
        f.write(line)

Run randobs manually in folder using the following arguments:

- obsweights.dat
- 1
- 500
- n
- obs-ensemble.csv (jcb)
- r
- enter

### Append IES settings to the pestfile

In [58]:
ies_settings = [
    '#PEST++ Variables\n'
    '++ies_num_reals(200)\n',
    '++ies_observation_ensemble(obs-ensemble.jcb)\n',
    '++parcov(param.unc)\n',
    '++ies_subset_size(5)\n',
    '++ies_num_threads(20)\n', # should not exceed the number of physical cores of the host machine on which the PESTPP-IES master instance is running
    '++ies_save_binary(true)\n',
    '++ies_ordered_binary(false)\n'
    '++ies_autoadaloc(true)\n'
]

In [59]:
for setting in ies_settings:
    pstfile.append(setting)

### Linuxify pstfile

In [60]:
pstfile = [i.replace('\\', '/') for i in pstfile]

In [62]:
pstfile[pstfile.index('* model command line\n') + 1] = './runmodel.sh\n'

### Change one-way observations to comply with ++ syntax

In [63]:
pstfile = [i.replace('>@dts', 'l_dts') for i in pstfile]
pstfile = [i.replace('>@of', 'l_of') for i in pstfile]

In [64]:
with open(os.path.join(pest_ws, 'ies-postcalib.pst'), 'w') as f:
    for line in pstfile:
        f.write(line)

### Linuxify uncertainty file

In [65]:
with open(os.path.join(pest_ws, 'param.unc'), 'r') as f:
    uncfile = f.readlines()

In [66]:
uncfile = [i.replace('\\', '/') for i in uncfile]

In [67]:
with open(os.path.join(pest_ws, 'param.unc'), 'w') as f:
    for line in uncfile:
        f.write(line)

In [ ]:
# rock n roll

In [ ]:
# Fråga Rui hur "Inequality Observations / One way observations ska viktas / anges i RANDOBS"